# Packages

This section contains the packages necessary for this project. 

In [ ]:
import math
import pandas as pd
import torch
from torch.utils.data import TensorDataset, DataLoader
import gpytorch
from tqdm.notebook import tqdm
import warnings
from torch.cuda import is_available as cuda_available, empty_cache
warnings.filterwarnings("ignore") #please be really sure about this one 

In [ ]:
#!pip freeze > requirements-01.txt

# Classes

This section contains the - very short - class implementation for the GP. 

In [ ]:
class MultitaskGPModel(gpytorch.models.ExactGP): #Multi Class GP with 6 (correlated) outputs
    def __init__(self, train_x, train_y, likelihood):
        super(MultitaskGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.MultitaskMean(
            gpytorch.means.ConstantMean(), num_tasks=6
        )
        
        self.covar_module = gpytorch.kernels.MultitaskKernel( #7 input variables, 6 outputs, simple 1-rank correlation for outputs
            gpytorch.kernels.RBFKernel(ard_num_dims=7), num_tasks=6, rank=1
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultitaskMultivariateNormal(mean_x, covar_x)

# Functions

This section contains function used in the GP training.

## Functions for: Data reformatting

In [ ]:
#filter_data_by_GEN: read full simulation data & filter for sequenced time points
def filter_data_by_GEN(data_path, values): 
    df = pd.read_csv(data_path, sep='\t', header=0) #read data 
    df['GEN'] = df['GEN'] - 1 #change from 1-based to 0-based
    filtered_df = df[df['GEN'].isin(values)] #filter for empirical available time points 
    return filtered_df

#select_columns_by_names: subset training data based on column names 
def select_columns_by_names(df, column_names):
    selected_columns = df[df['GEN']== 0] #for parameter extraction: removes duplicated entries
    selected_columns = selected_columns[column_names]
    selected_columns = selected_columns.to_numpy(dtype = "float")
    return torch.from_numpy(selected_columns).float().contiguous()


#get_CN_for_GEN get CN ( = response) for specific generation GEN
def get_CN_for_GEN(df, GEN):
    filtered_df = df[df['GEN'] == GEN] #filter for specific GEN
    if filtered_df.empty:
        return None 
    else:
        CN = filtered_df['m'].to_numpy(dtype = "float") #m contains avg. P-ele CN/haplotype across n simulations
        return torch.from_numpy(CN).float().contiguous().flatten()

#prep_data: re-formatting of SLiMULATION output for GP 
def prep_data(data_path, input_params, time_points):
    df = filter_data_by_GEN(data_path=data_path, values=time_points) #filter data
    df_x = select_columns_by_names(df = df, column_names=input_params) #extract input parameters 
    y_list = [] 

    for gen in time_points[1:]: #exclude gen = 0 (no predictions); extract y-values (= CN | GEN)
        CN_values = get_CN_for_GEN(df, gen)
        y_list.append(CN_values)
    y = torch.stack(y_list, dim=-1) #reformat y-values 
    if cuda_available():
        df_x, y = df_x.cuda(), y.cuda()
    return df_x, y #return reformatted input & output data 

#calculate_rmse: calculate RMSE 
def calculate_rmse(predictions, observations):
    squared_error = (predictions - observations)**2
    mse = squared_error.mean(dim=0) #MSE across first dimension (= input parameters)
    rmse_per_timepoint = torch.sqrt(mse)
    overall_rmse = torch.sqrt(squared_error.mean()) #overall RMSE
    return rmse_per_timepoint, overall_rmse


## Functions for: GP handling

In [ ]:
#trainGP: train Gaussian process for <iterationcount> iterations, using learning rate <learning_rate> 
def trainGP(model, likelihood, train_x, train_y, learning_rate, iterationcount):
    model.train() #set to training modus
    likelihood.train()

    # Use the adam optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr= learning_rate)  # Includes GaussianLikelihood parameters

    # "Loss" for GPs - the marginal log likelihood
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

    for i in tqdm(range(iterationcount), desc="Iterations comple on this training round.", leave=False): #train for <iterationcount> interations 
        if cuda_available():
            torch.cuda.empty_cache()
        optimizer.zero_grad()
        output = model(train_x)
        loss = -mll(output, train_y)
        loss.backward()
        #if (i + 1) % 500==0 or i == 0: #output for tracking of loss on training data 
            #print('Iter %d/%d - Loss: %.3f' % (i + 1, iterationcount, loss.item()))
        if i == (iterationcount-1):
            lossCount = loss.item() #save Loss at end of training
        optimizer.step()

    model.eval() #set back to evaluation mode 
    likelihood.eval()
    return model, likelihood, lossCount #return trained model & loss at the end of trianing

#GPpredict: predict y|x with GP
def GPpredict(df, model, likelihood):
    with torch.no_grad(), gpytorch.settings.fast_pred_var():
       predictions = likelihood(model(df))
       mean = predictions.mean
       lower, upper = predictions.confidence_region() 
    return lower, mean, upper
    # loader = DataLoader(TensorDataset(df), batch_size=128, shuffle=False)
    # mean, lower, upper = torch.tensor([0.]), torch.tensor([0.]), torch.tensor([0.])
    # with torch.no_grad(), gpytorch.settings.fast_pred_var():
    #     batchnum = 1
    #     for batch in loader:
    #         if cuda_available():
    #             torch.cuda.empty_cache()
    #         print(batchnum)
    #         batchnum +=1
    #         predictions = likelihood(model(batch[0]))
    #         cur_mean = predictions.mean
    #         if cuda_available():
    #             mean = torch.cat([mean, cur_mean.cpu()])
    #             cur_lower, cur_upper = predictions.confidence_region()
    #             lower = torch.cat([lower, cur_lower.cpu()])
    #             upper = torch.cat([upper, cur_upper.cpu()])
    #         else:
    #             mean = torch.cat([mean, cur_mean])
    #             cur_lower, cur_upper = predictions.confidence_region()
    #             lower = torch.cat([lower, cur_lower])
    #             upper = torch.cat([upper, cur_upper])
    # return lower[1:], mean[1:], upper[1:]

# Variables

This section contains global variables. If you use the provided file structure, <i>INVASION_TYPE</i> is the only variable that needs to be changed.  

In [ ]:
INVASION_TYPE = "established" #type of invasion (early, established)
TRAIN_SET_NUM_POINTS = 2500

In [ ]:

ESTABLISHED_TP = [0, 10, 15, 20, 25, 30, 60] #sequenced time points: established P-element invasion (do not change)
EARLY_TP = [0, 10, 20, 30, 40, 50, 60] #sequenced time points: early P-element invasion (do not change)

if INVASION_TYPE not in ["early", "established"]: #time points used for GP 
   raise ValueError("Specify the type of invasion \"early\", \"established\"")
else:
   if INVASION_TYPE == "early":
        TIME_POINTS = EARLY_TP
   else: 
        TIME_POINTS = ESTABLISHED_TP
        
PATH_TRAIN = f"../data/{INVASION_TYPE}/train-LHS-{TRAIN_SET_NUM_POINTS}-allGen-betaDist-varReg-varP-ExcEvents-rescaled.txt" #path training data
PATH_VAL = f"../data/{INVASION_TYPE}/val-LHS-10000-allGen-betaDist-varReg-varP-ExcEvents-rescaled.txt" #path validation data 
PATH_TEST = f"../data/{INVASION_TYPE}/test-LHS-10000-allGen-betaDist-varReg-varP-ExcEvents-rescaled.txt" #path test data 

INPUT_PARAM = ['trans_prob', 'sel_alpha', 'sel_beta', 'l_pi', 'p_te', 'p_exc', 'h'] #model parameters


LEARNING_RATE = 0.01 #GP: learning rate 
ITERATION_COUNT = 100 #GP: number of iterations per training round 
TRAINING_ROUNDS = 50 #GP: number of training rounds: total amount of training = ITERATION_COUNT X TRAINING_ROUNDS

# GP

## Set up Data

In [ ]:
train_x, train_y = prep_data(data_path=PATH_TRAIN, input_params=INPUT_PARAM, time_points=TIME_POINTS) #prep training data 
val_x, val_y = prep_data(data_path=PATH_VAL, input_params=INPUT_PARAM, time_points=TIME_POINTS) #prep validation data 
test_x, test_y = prep_data(data_path=PATH_TEST, input_params=INPUT_PARAM, time_points=TIME_POINTS) #prep test data 

for i in train_x, train_y, val_x, val_y, test_x, test_y : #sanity checks: format... X = 10,000 rows x 7 model parameters; Y = 10,000 rows x 6 P-element copy numbers (time series)
    print(i.shape)
    print(i.dtype)

## Set up GP

In [ ]:
likelihood = gpytorch.likelihoods.MultitaskGaussianLikelihood(num_tasks=6) #set-up GP 
model = MultitaskGPModel(train_x, train_y, likelihood)
if cuda_available():
    print("Using GPU acceleration")
    likelihood, model = likelihood.cuda(), model.cuda()

## Training 

This section contains the actual training of the GP. After Training, there should be <i>TRAINING_ROUNDS</i> model snapshots in the <i>models/INVASIONTYPE</i> directory plus a <i>stats-GP.txt</i> file that contains (among other things) the RMSEs for all model snapshots for the training and validation data. 

In [ ]:
for i in tqdm(range(TRAINING_ROUNDS), desc="Total training rounds"):
        if i == 0 : #if new training attempt, create new file with header
                with open(f"../models/{INVASION_TYPE}/stats-GP.txt", 'w') as file:
                    myheader = "invasion_type\tround\ttraining_loss\ttraining_rmse\tvalidation_rmse\tneg_count_val\n"
                    file.write(myheader)
                    file.close()
        with open(f"../models/{INVASION_TYPE}/stats-GP.txt", 'a') as file:
            model, likelihood, loss = trainGP(model=model, likelihood=likelihood,train_x=train_x, train_y=train_y, iterationcount=ITERATION_COUNT, learning_rate=LEARNING_RATE) #train
            if cuda_available():
                torch.cuda.empty_cache()
            model_snapshot = model
            if cuda_available():
                model = model.cpu()  # Save a cpu model even if using GPU acceleration.
                torch.save(model.state_dict(), f"../models/{INVASION_TYPE}/P-GP-{i}.pth") #save model snapshot 
                model = model.cuda()
            else:
                torch.save(model.state_dict(), f"../models/{INVASION_TYPE}/P-GP-{i}.pth") #save model snapshot 
            _, pred_train, _ = GPpredict(df = train_x, model=model, likelihood=likelihood) #predicted output | training data 
            _, rmse_train = calculate_rmse(predictions=pred_train, observations=train_y) #RMSE training data 
            _, pred_val,_ = GPpredict(df=val_x, model=model, likelihood=likelihood) #predicted output | validation data 
            _, rmse_val = calculate_rmse(predictions=pred_val, observations=val_y) #RMSE validation data 
            neg_count = pred_val < 0
            neg_count = neg_count.sum().item() #number of negative pred. values 

            myoutput=f"{INVASION_TYPE}\t{i}\t{loss}\t{rmse_train}\t{rmse_val}\t{neg_count}\n" #generate output 
            file.write(myoutput + '\n')

## Prediction
Predict P element invasion dynamics using the GP on the test data and store the results. After running this section, there should be a <i>P-GP-predict.txt</i> file in the <i>pred/INVASION_TYPE</i> directory

In [ ]:
stats = pd.read_csv(f"../models/{INVASION_TYPE}/stats-GP.txt", sep='\t', header=0) #load model with lowest RMSE on validation data 
min_id= stats['validation_rmse'].idxmin()
toload = f"../models/{INVASION_TYPE}/P-GP-{stats['round'][min_id]}.pth"
print(toload)
model.load_state_dict(torch.load(toload))

lower_val, pred_val, upper_val = GPpredict(df=val_x, model=model, likelihood=likelihood) #sanity check 
_, rmse_val = calculate_rmse(predictions=pred_val, observations=val_y)
print(rmse_val)
print(stats['validation_rmse'][min_id])

lower_test, pred_test, upper_test = GPpredict(df = test_x, model=model, likelihood=likelihood) #predicted output | test data 
rmse_test_TP, rmse_test = calculate_rmse(predictions=pred_test, observations=test_y) #RMSE test data 
print(rmse_test_TP)
print(rmse_test)

In [ ]:
test_df = filter_data_by_GEN(data_path=PATH_TEST, values=TIME_POINTS)
test_df['pred'] = float('nan')
test_df['lower'] = float('nan')
test_df['upper'] = float('nan')

if cuda_available():
    lower_test, pred_test, upper_test = lower_test.cpu(), pred_test.cpu(), upper_test.cpu()

for gen in range(1, len(TIME_POINTS)): #add predicted values & credible interval (attention: no prediction for generation 0)
    test_df.loc[test_df['GEN'] == TIME_POINTS[gen], 'pred'] = pred_test[:, (gen-1)].numpy()
    test_df.loc[test_df['GEN'] == TIME_POINTS[gen], 'lower'] = lower_test[:, (gen-1)].numpy()
    test_df.loc[test_df['GEN'] == TIME_POINTS[gen], 'upper'] = upper_test[:, (gen-1)].numpy()

test_df.to_csv(f"../pred/{INVASION_TYPE}/P-GP-predict.txt", sep='\t', index=False) #save predictions